In [45]:
#Importar bibliotecas do sistemas
import sys
import os

# Caminho absoluto até a pasta src
sys.path.append(os.path.abspath("../src"))

#Importar libs do python
import pandas as pd 
import numpy as np 
import math
from pyautocad import Autocad, APoint 

#Importar libs do programa 
import design_functions
from v_p_viga_sobre_pilar.v_p_viga_sobre_pilar import *
from v_p_chapa_cabeca.v_p_chapa_cabeca import parametro_b,espessura_solda
from draw_autocad.draw_autocad_figures import *
from materials import * 

In [46]:
perfil_pilar = W_610x174_0

aço_escolhido = ASTM_A36

aco_chapa = ASTM_A36

solda_escolhida =E70XX

perfil_pilar.inercias()
perfil_pilar.material(aço_escolhido)


##### Cálculo da disposição da chapa, dos parafusos e do diâmetro minimo

In [47]:
M = 650600 #kN.mm 

V = 720 #kN

T = 480 #kN

parafuso = ASTM_A325
parafuso.prop_geometricas(rosca = 1,planos_de_corte =1)

enrijecedor = 1

S = dim_chapa_pilar(M,V,T,aco_chapa,enrijecedor,perfil_pilar,parafuso,gamma)

In [48]:
S

[11,
     parafuso  x (mm)  y (mm)
 0          1      40    25.0
 1          2     285    25.0
 2          3      40   106.6
 3          4     285   106.6
 4          5      40   170.7
 5          6     285   170.7
 6          7      40   234.8
 7          8     285   234.8
 8          9      40   298.9
 9         10     285   298.9
 10        11      40   363.0
 11        12     285   363.0
 12        13      40   427.1
 13        14     285   427.1
 14        15      40   491.2
 15        16     285   491.2
 16        17      40   555.3
 17        18     285   555.3
 18        19      40   619.4
 19        20     285   619.4
 20        21      40   701.0
 21        22     285   701.0,
 25.4,
 15.875]

In [49]:
parafuso.diametro_mm

19.049999999999997

In [50]:
chapa=S[2]
ver_parafuso=S[3]
exp=S[4]
if enrijecedor ==1:
    enj = S[5]

In [51]:
exp

25.4

##### Cálculo da espessura do filete de solda na alma e na mesa

In [52]:
espessura = espessura_solda(M,T,V,solda_escolhida,perfil_pilar,exp,True,gamma)

esp = math.ceil(espessura) #mm

esp

6

##### Abertura do projeto no pyautocad

In [53]:
acad = Autocad(create_if_not_exists=True)
acad.prompt("Hello, Autocad from Python\n")
print(acad.doc.Name)

Hello, Autocad from Python

Drawing1.dwg


In [54]:
limpar_desenho(acad)

✅ Desenho limpo com sucesso.


In [55]:
pontos_hexagono = gerar_pontos_hexagono(parafuso.diametro_mm)

In [56]:
# Chamando a função para desenhar a chapa 3D
objetos_chapa = criar_chapa_3d(acad, chapa.df, exp)

In [57]:
objetos_parafusos=[]
for i in range(ver_parafuso.shape[0]):
    x_centro = ver_parafuso.iat[i, 1]
    y_centro = ver_parafuso.iat[i, 2]

    # Adicionar circunferência no ponto
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,exp), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    obj = acad.model.AddCircle(APoint(x_centro, y_centro,0), parafuso.diametro_mm / 2)
    objetos_parafusos.append(obj)
    # Transladar hexágono para o ponto atual
    hexagono_transladado = transladar_pontos(pontos_hexagono, x_centro, y_centro, exp)

    for j in range(len(hexagono_transladado) - 1):
        p1 = APoint(*hexagono_transladado[j])
        p2 = APoint(*hexagono_transladado[j + 1])
        obj = acad.model.AddLine(p1, p2)
        objetos_parafusos.append(obj)

In [58]:
base_perfil= min(ver_parafuso['y (mm)'])+ parametro_b(parafuso.diametro_mm)

In [59]:
objetos_secao_perfil = desenhar_secao_perfil(acad, perfil_pilar, (chapa.B / 2) - (perfil_pilar.b_f / 2), posicao_y=base_perfil, altura_z=exp)

In [60]:
if enrijecedor == 1:
    desenhar_enrijecedores(acad, (0,0,exp),base_perfil ,chapa, perfil_pilar, ver_parafuso, parafuso.diametro_mm, enj)